In [4]:
import os
import json
import math
from pathlib import Path

# ==========================================
# 1. KONFIGURASJON
# ==========================================
TARGET_DIR = Path("./artifacts")        # Mappen vi skal skanne
OUTPUT_FILENAME = "repo_data_dump.txt"  # Resultatfilen

ALLOWED_EXTENSIONS = {'.json', '.csv'}

# --- SAMPLING INNSTILLINGER ---
# Dette styrer "S-kurve" logikken.
TARGET_TOTAL_ITEMS = 30   # Hvor mange elementer vi totalt vil vise fra en lang liste
WINDOW_SIZE = 3           # Hvor mange elementer vi viser sammenhengende i hver "chunk"
MAX_JSON_DEPTH = 4        # Hvor dypt ned i JSON-strukturen vi går

# ==========================================
# 2. LOGIKK FOR DISTRIBUERT SAMPLING
# ==========================================

def get_sampled_list(full_list):
    """
    Tar en lang liste og returnerer en samplet versjon som viser data
    jevnt fordelt utover hele listen (start, midt, slutt osv).
    """
    total_len = len(full_list)
    
    # Hvis listen er kort nok, vis alt
    if total_len <= TARGET_TOTAL_ITEMS:
        return full_list
    
    sampled_output = []
    
    # Beregn hvor mange "vinduer" (grupper) vi har plass til
    num_windows = max(2, TARGET_TOTAL_ITEMS // WINDOW_SIZE)
    
    # Beregn steget (hvor langt vi hopper mellom hvert vindu)
    step = (total_len - WINDOW_SIZE) / (num_windows - 1)
    
    last_idx = -1
    
    for i in range(num_windows):
        # Finn start-indeksen for dette vinduet
        start_idx = int(i * step)
        
        # Unngå overlapp eller å gå bakover (sikkerhetsnett)
        if start_idx <= last_idx:
            start_idx = last_idx + 1
        
        end_idx = min(start_idx + WINDOW_SIZE, total_len)
        
        # Hvis vi hoppet over data, legg til en indikator
        if start_idx > last_idx + 1 and last_idx != -1:
            skipped_count = start_idx - (last_idx + 1)
            sampled_output.append(f"... [skipped {skipped_count} items] ...")
        
        # Legg til selve dataene i vinduet
        chunk = full_list[start_idx:end_idx]
        sampled_output.extend(chunk)
        
        last_idx = end_idx - 1
        
        # Hvis vi har nådd slutten, stopp
        if last_idx >= total_len - 1:
            break
            
    return sampled_output

def compress_json_recursive(data, depth=0):
    """
    Rekursiv JSON-behandling som bruker sampling på lister.
    """
    if depth >= MAX_JSON_DEPTH:
        return "..." 
    
    if isinstance(data, dict):
        new_dict = {}
        keys = list(data.keys())
        
        # For dictionaries viser vi vanligvis alle nøkler med mindre det er ekstremt mange,
        # fordi nøkler ofte er unike identifikatorer.
        # Men vi sampler innholdet i verdiene.
        limit_keys = 20 # Sikkerhetsgrense for dicts med tusenvis av nøkler
        
        for i, k in enumerate(keys):
            if i >= limit_keys:
                new_dict[f"..."] = f"({len(keys) - limit_keys} more keys hidden)"
                break
            new_dict[k] = compress_json_recursive(data[k], depth + 1)
        return new_dict
    
    elif isinstance(data, list):
        # Her skjer magien: Vi sampler listen i stedet for å kutte den tvert
        # Vi rekursivt komprimerer elementene vi beholder også
        sampled_raw = get_sampled_list(data)
        
        # Kjør rekursjon på elementene vi beholdt (hvis de er lister/dicts)
        processed_list = []
        for item in sampled_raw:
            if isinstance(item, str) and item.startswith("... [skipped"):
                processed_list.append(item) # Behold skip-markøren
            else:
                processed_list.append(compress_json_recursive(item, depth + 1))
                
        return processed_list
    
    return data # Returner primitive typer som de er

def process_csv_content(text_content):
    """
    Sampler CSV-linjer fordelt utover filen.
    """
    lines = text_content.splitlines()
    if not lines: return ""
    
    header = lines[0]
    data_rows = lines[1:]
    
    if not data_rows: return header
    
    # Bruk samme sampling-logikk på radene
    sampled_rows = get_sampled_list(data_rows)
    
    output = [header]
    for row in sampled_rows:
        output.append(str(row)) # row er en streng eller skip-markør
        
    return "\n".join(output)

# ==========================================
# 3. FILBEHANDLING OG MAIN
# ==========================================

def get_directory_tree(root_path):
    """Lager en enkel tekst-trestruktur av filene vi fant."""
    lines = [f"=== ARTIFACTS STRUCTURE ({root_path}) ==="]
    all_files = sorted([p for p in root_path.rglob("*") if p.is_file() and p.suffix.lower() in ALLOWED_EXTENSIONS])
    
    if not all_files:
        return "No .json or .csv files found in artifacts folder.\n"

    for path in all_files:
        rel_path = path.relative_to(root_path.parent)
        lines.append(f"- {rel_path}")
    lines.append("")
    return "\n".join(lines)

def process_file(file_path):
    header = f"\n{'='*80}\nFILE: {file_path}\n{'='*80}\n"
    try:
        if file_path.suffix.lower() == '.json':
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            compressed = compress_json_recursive(data)
            return header + json.dumps(compressed, indent=2) + "\n"
            
        elif file_path.suffix.lower() == '.csv':
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
            processed = process_csv_content(content)
            return header + processed + "\n"
            
    except Exception as e:
        return header + f"[ERROR: {e}]\n"
    return ""

def main():
    if not TARGET_DIR.exists():
        print(f"Feil: Fant ikke mappen '{TARGET_DIR}'.")
        return

    full_output = []
    print(f"Scanner '{TARGET_DIR}' (rekursivt) med distribuert sampling...")
    
    # 1. Struktur
    full_output.append(get_directory_tree(TARGET_DIR))
    
    # 2. Innhold
    count = 0
    for file_path in sorted(TARGET_DIR.rglob("*")):
        if file_path.is_file() and file_path.suffix.lower() in ALLOWED_EXTENSIONS:
            full_output.append(process_file(file_path))
            count += 1
            print(f"  -> Behandlet: {file_path.name}")

    # 3. Lagre
    with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
        f.write("".join(full_output))
        
    print(f"\nFerdig! {count} filer lagret til '{OUTPUT_FILENAME}'.")

if __name__ == "__main__":
    main()

Scanner 'artifacts' (rekursivt) med distribuert sampling...
  -> Behandlet: training_stats-checkpoint.json
  -> Behandlet: training_stats.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results-checkpoint.json
  -> Behandlet: weights_history-checkpoint.json
  -> Behandlet: results.json
  -> Behandlet: weights_history.json
  -> Behandlet: results.json
  -> Behandlet: weights_history.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results-checkpoint.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: results.json
  -> Behandlet: r